# Similarity Modeling 1/2: Recognizing Pigs with Deep Learning

## Student
Gallus Huber (51905700)

## Timesheet
2022-10-16 09:00 - 12:30 SIM 1 & 2 <br />
2022-10-16 15:00 - 17:00 SIM 3 <br />
2022-10-18 10:00 - 13:00 SIM 4 & 5 <br />
2022-10-27 20:00 - 22:00 SIM 6 <br />
2022-10-28 10:00 - 14:00 SIM 7 & 8 & 9 <br />
2022-11-02 08:00 - 14:00 Writing abstracts <br />
2022-11-10 14:00 - 15:00 Meeting <br />
2022-11-24 16:00 - 17:30 Setup <br />
2022-11-26 09:00 - 12:00 Implementation data preparation <br />
2022-11-27 10:00 - 10:30 Meeting <br />
2022-11-27 10:30 - 15:00 Implementation model <br />
2022-12-01 16:00 - 20:00 Training of model <br />
2022-12-03 14:00 - 18:00 Improvements <br />
2022-12-18 14:00 - 18:00 Improvements & training <br />
2022-12-27 18:20 - 21:00 Improvements

## Required libraries and installs

In [ ]:
!cat requirements-dl-video.txt
!pip install -r requirements-dl-video.txt

## Imports

In [ ]:
import os
import glob
import pathlib
import collections

import pandas as pd
import numpy as np

import cv2 as cv2
import matplotlib.pyplot as plt

import sklearn.model_selection as ms

import tensorflow as tf

import keras.applications as appl
import keras.models as models
import keras.layers as layers
import keras.utils as utils
import keras.callbacks as cb
import keras.optimizers as optimizers

import visualkeras

## Constants

In [ ]:
# path to ground truth and videos
DATA_PATH = "/content/drive/MyDrive/data/"
GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "sim12-ground-truth-muppets")
VIDEO_PATH = os.path.join(DATA_PATH, "video")

# path to train data
TRAINING_PATH = os.path.join(VIDEO_PATH, 'training_data')
VALIDATION_PATH = os.path.join(VIDEO_PATH, 'validation_data')
TESTING_PATH = os.path.join(VIDEO_PATH, 'testing_data')

# path to save and load trained models
MODEL_PATH = "/content/drive/MyDrive/data/models"

# Project Structure

In [ ]:
# create dirs for training
train_path_0 = os.path.join(TRAINING_PATH, '0')
train_path_1 = os.path.join(TRAINING_PATH, '1')

# create dirs for validation
valid_path_0 = os.path.join(VALIDATION_PATH, '0')
valid_path_1 = os.path.join(VALIDATION_PATH, '1')

# create dirs for testing
test_path_0 = os.path.join(TESTING_PATH, '0')
test_path_1 = os.path.join(TESTING_PATH, '1')

In [ ]:
# create dirs for training
if not os.path.exists(TRAINING_PATH):
    os.makedirs(TRAINING_PATH)
if not os.path.exists(train_path_0):
    os.makedirs(train_path_0)
if not os.path.exists(train_path_1):
    os.makedirs(train_path_1)

# create dirs for validation
if not os.path.exists(VALIDATION_PATH):
    os.makedirs(VALIDATION_PATH)
if not os.path.exists(valid_path_0):
    os.makedirs(valid_path_0)
if not os.path.exists(valid_path_1):
    os.makedirs(valid_path_1)

# create dirs for testing
if not os.path.exists(TESTING_PATH):
    os.makedirs(TESTING_PATH)
if not os.path.exists(test_path_0):
    os.makedirs(test_path_0)
if not os.path.exists(test_path_1):
    os.makedirs(test_path_1)

# create dir for models
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

# 1. Data Preparation

## 1.1 List all videos

In [ ]:
videos = glob.glob(VIDEO_PATH + '/*.avi')

videos_tmp = []
for video in videos:
    videos_tmp.append(pathlib.Path(video).name)
videos = videos_tmp

videos

## 1.2 Load ground truth

### 1.2.1 Read ground truth from CSV

In [ ]:
ground_truth = dict()

# read in corresponding ground truth
for video in videos:
        file_name = glob.glob(GROUND_TRUTH_PATH + '/*' + os.path.splitext(video)[0] + '*.csv')
        csv = pd.read_csv(os.path.join(file_name[0]), sep=";")
        ground_truth[video] = csv

### 1.2.2 Plot the ground truth

In [ ]:
def plot_gt(gt, column):
    plot = plt.figure(figsize=(10, 4))

    # define subplot
    subplot = plot.add_subplot(111)
    subplot.set_title(f"Occurrences in Frames")

    # define x axis
    subplot.set_xlabel('Frames')

    # mark occurrences
    frame_indicator = np.zeros(len(gt['Frame_number']))
    frame_indicator[gt[gt[column] == 1].index] = 1

    subplot.plot(gt[column].index, frame_indicator)
    plt.show()

for video in videos:
    plot_gt(ground_truth[video], "Pigs")

## 1.3 Load video data

### 1.3.1 Data structure to store splits

In [ ]:
train_data = dict()
valid_data = dict()
test_data = dict()

### 1.3.2 Create splits
The ground truth is split into training set, validation set and testing set.

In [ ]:
for video in videos:
    train_data[video], test_data[video] = ms.train_test_split(ground_truth[video], test_size=0.2, shuffle=True, random_state=42)
    train_data[video], valid_data[video] = ms.train_test_split(train_data[video], test_size=0.25, random_state=42)

### 1.3.3 Get frames of video
The frames are written to the corresponding directory based on the splits and ground truth.

In [ ]:
def retrieve_frames(video_file):
    # open a videoCap for this video
    video_cap = cv2.VideoCapture(os.path.join(VIDEO_PATH, video_file))

    # used to identify frame in ground truth
    frame_nr = 0

    # read all frames of video
    while True:
        # get one frame
        success_read, frame_read = video_cap.read()

        # no more frames available
        if not success_read:
            break

        # create filename
        filename = 'frame_%d.jpg' % frame_nr

        # write frames to dir
        if frame_nr in train_data[video_file].Frame_number:
            if (train_data[video_file][train_data[video_file].Frame_number == frame_nr].Pigs == 0).bool():
                cv2.imwrite(os.path.join(train_path_0, filename), frame_read)
            else:
                cv2.imwrite(os.path.join(train_path_1, filename), frame_read)

        elif frame_nr in valid_data[video_file].Frame_number:
            if (valid_data[video_file][valid_data[video_file].Frame_number == frame_nr].Pigs == 0).bool():
                cv2.imwrite(os.path.join(valid_path_0, filename), frame_read)
            else:
                cv2.imwrite(os.path.join(valid_path_1, filename), frame_read)

        elif frame_nr in test_data[video_file].Frame_number:
            if (test_data[video_file][test_data[video_file].Frame_number == frame_nr].Pigs == 0).bool():
                cv2.imwrite(os.path.join(test_path_0, filename), frame_read)
            else:
                cv2.imwrite(os.path.join(test_path_1, filename), frame_read)

        # go to next frame
        frame_nr += 1


for video in videos:
    retrieve_frames(video)
    print(f"Loaded frames of {video} successfully")

# 2 Model

## 2.1 Build model

### 2.1.1 Create the model

In [ ]:
# create base model -> pretrained model
base_model = appl.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add base model
head_model = base_model.output

# additional hidden layer
head_model = layers.Conv2D(512, kernel_size=(7, 7), activation="relu", padding='same')(head_model)
head_model = layers.Conv2D(512, kernel_size=(7, 7), activation="relu", padding='same')(head_model)
head_model = layers.Conv2D(512, kernel_size=(7, 7), activation="relu", padding='same')(head_model)

# create output layer
head_model = layers.MaxPooling2D(pool_size=(2, 2))(head_model)
head_model = layers.Flatten()(head_model)
head_model = layers.Dense(1, activation="sigmoid")(head_model)

# create model
model = models.Model(base_model.input, head_model)

# base model should not be trained
for layer in base_model.layers:
	layer.trainable = False

model.summary()

### 2.1.2 Plot the model


In [ ]:
# set colors for each used layer
color_map = collections.defaultdict(dict)
color_map[layers.InputLayer]['fill'] = '#7D670C'
color_map[layers.Conv2D]['fill'] = '#060A7D'
color_map[layers.MaxPooling2D]['fill'] = '#7C0400'
color_map[layers.Flatten]['fill'] = '#7D066F'
color_map[layers.Dense]['fill'] = '#0C7D34'
color_map[layers.Dropout]['fill'] = '#808080'

# plot the model
visualkeras.layered_view(model, legend=True, color_map=color_map)

## 2.2 Compile model

In [ ]:
metrics = [tf.keras.metrics.AUC(name='prc', curve='PR', num_thresholds=200),
                   tf.keras.metrics.Recall(name="recall"), 
                   tf.keras.metrics.Precision(name="precision")]

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), 
              metrics=metrics)

## 2.3 Train model

### 2.3.1 Load training and validation sets

In [ ]:
# load training set
train_ds = utils.image_dataset_from_directory(
    directory=TRAINING_PATH,
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# load validation set
validation_ds = utils.image_dataset_from_directory(
    directory=VALIDATION_PATH,
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

### 2.3.2 Analyze class imbalance

In [ ]:
train_0 = len(os.listdir(train_path_0))
train_1 = len(os.listdir(train_path_1))

print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        train_1, 100 * float(train_1) / len(train_ds.file_paths)
    )
)

weight_for_0 = 1.0 / train_0
weight_for_1 = 1.0 / train_1

class_weight = {0: weight_for_0, 1: weight_for_1}

### 2.3.3 Normalize


In [ ]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
validation_ds = validation_ds.map(lambda x, y: (normalization_layer(x), y))

### 2.3.4 Train model

In [ ]:
# callback to avoid overfitting and storing best model
callbacks = [cb.EarlyStopping(monitor='val_loss', patience=10),
             cb.ModelCheckpoint(filepath=os.path.join(MODEL_PATH, 'video-dl'), monitor='val_prc', mode='max', save_best_only=True, save_weights_only=True)]

# train model
with tf.device('/device:GPU:0'):
  history = model.fit(train_ds, epochs=100, validation_data=validation_ds, callbacks=callbacks, class_weight=class_weight)

### 2.3.5 Plot training results

In [ ]:
history = history.history
epochs = np.arange(len(history["loss"]))

fig, axs = plt.subplots(2, 2, figsize=(15, 10))
axs[0, 0].plot(epochs, history["loss"], label="Training")
axs[0, 0].plot(epochs, history["val_loss"], label="Validation")
axs[0, 0].set_title("Loss (Binary Cross Entropy)")
axs[0, 0].legend()

axs[0, 1].plot(epochs, history["prc"], label="Training")
axs[0, 1].plot(epochs, history["val_prc"], label="Validation")
axs[0, 1].set_title("Precision/Recall AUC")
axs[0, 1].legend()

axs[1, 0].plot(epochs, history["precision"], label="Training")
axs[1, 0].plot(epochs, history["val_precision"], label="Validation")
axs[1, 0].set_title("Precision")
axs[1, 0].legend()

axs[1, 1].plot(epochs, history["recall"], label="Training")
axs[1, 1].plot(epochs, history["val_recall"], label="Validation")
axs[1, 1].set_title("Recall")
axs[1, 1].legend()

plt.tight_layout()
plt.show()

# 3 Results

## 3.1 Load best model

In [ ]:
model = models.load_model(os.path.join(MODEL_PATH, 'video-dl'))

## 3.2 Evaluate model

In [ ]:
# load testing set
test_ds = utils.image_dataset_from_directory(
    directory=TESTING_PATH,
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# normalize testing set
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# evaluate the model
results = model.evaluate(test_ds, batch_size=128)